<a href="https://colab.research.google.com/github/AsadbekTuronov/data_science_and_AI/blob/main/Aviakompaniya2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'avia2dataset:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F5869780%2F9617962%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20241014%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20241014T001348Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Da2ec4a2f3e14f946f0130c4d12e12fd0de86e977938e3b9278953a56a584351eba6b8e8d07c99e204394cab8ef9361134c8a7e6c8e77d4edda4a55f9577b03ea04465b95c411bb3c705d7780cbc57f0c4e7c5788b1ee998542a9502731def4e2abd1a755f9c8db177f293d9c43fb10c80efe0d0833ad95fc6646789543f087d3512b16a98fcba0b97345136d577132535fa356fe8e01ecc17dce88e539e8af2573d0a655274b00d14de34ad72d47b2f8094e10c9b84a6953a32b30e1acccb1aa3851b7c86a009615d0db03e6db05db353a51c152fd5a59da79d34757213c00f0d987bbc49216607cbde5ec0f032f638fd9241c31497ca08c8f8f7556bb0e2d00'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 250323 bytes downloaded
Downloaded and uncompressed: avia2dataset
Data source import complete.


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/avia2dataset/sample_submission.csv
/kaggle/input/avia2dataset/test_dataset.csv
/kaggle/input/avia2dataset/train_dataset.csv


In [4]:
df = pd.read_csv("/kaggle/input/avia2dataset/train_dataset.csv")

In [5]:
df.head()

,id,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,...,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,1,Male,disloyal Customer,33,Business travel,Eco,571,2,3,2,...,4,3,1,3,4,3,4,10,3.0,0
1,2,Female,Loyal Customer,49,Business travel,Business,1431,4,1,4,...,5,5,5,5,3,5,3,0,0.0,1
2,3,Female,Loyal Customer,43,Business travel,Eco,867,1,4,4,...,1,1,1,1,1,1,2,0,18.0,0
3,4,Female,Loyal Customer,27,Business travel,Business,1550,3,3,3,...,2,4,4,5,5,4,2,0,0.0,1
4,5,Male,Loyal Customer,11,Personal Travel,Eco,526,3,4,3,...,4,5,2,5,3,5,4,0,10.0,0


In [133]:
df.describe()

,id,Age,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Gate location,Food and drink,Online boarding,Seat comfort,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
count,10000.00000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,9972.000000,10000.000000
mean,5000.50000,39.375100,1229.556200,2.780200,3.065100,2.797800,2.981000,3.214500,3.349100,3.49140,3.402900,3.454100,3.415700,3.676200,3.360600,3.680100,3.311100,14.429900,14.709888,0.500000
std,2886.89568,14.897959,1022.281958,1.362745,1.527316,1.421168,1.283278,1.333217,1.352779,1.31122,1.331822,1.274777,1.308765,1.189746,1.258462,1.175118,1.311217,39.659674,40.046355,0.500025
min,1.00000,7.000000,31.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.00000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,2500.75000,27.000000,421.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,3.00000,2.000000,3.000000,2.000000,3.000000,3.000000,3.000000,2.000000,0.000000,0.000000,0.000000
50%,5000.50000,40.000000,863.500000,3.000000,3.000000,3.000000,3.000000,3.000000,4.000000,4.00000,4.000000,4.000000,4.000000,4.000000,3.000000,4.000000,3.000000,0.000000,0.000000,0.500000
75%,7500.25000,51.000000,1806.750000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,5.00000,5.000000,4.000000,5.000000,5.000000,4.000000,5.000000,4.000000,12.000000,12.000000,1.000000
max,10000.00000,80.000000,4983.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.00000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,921.000000,924.000000,1.000000


In [136]:
df[df.describe().columns].corrwith(df["satisfaction"])

,0
id,0.002931
Age,0.144302
Flight Distance,0.298703
Inflight wifi service,0.277395
Departure/Arrival time convenient,-0.067638
Ease of Online booking,0.164662
Gate location,0.007169
Food and drink,0.207854
Online boarding,0.504986
Seat comfort,0.342294


**pipeline**

In [145]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, FunctionTransformer, LabelEncoder
from sklearn.impute import SimpleImputer

In [101]:
for i in ["Gender", "Customer Type", "Type of Travel", "Class"]:
    print(df[i].value_counts())
    print()

Gender
Female    5169
Male      4831
Name: count, dtype: int64

Customer Type
Loyal Customer       8307
disloyal Customer    1693
Name: count, dtype: int64

Type of Travel
Business travel    7079
Personal Travel    2921
Name: count, dtype: int64

Class
Business    5048
Eco         4251
Eco Plus     701
Name: count, dtype: int64



In [26]:
df.columns

Index(['id', 'Gender', 'Customer Type', 'Age', 'Type of Travel', 'Class',
       'Flight Distance', 'Inflight wifi service',
       'Departure/Arrival time convenient', 'Ease of Online booking',
       'Gate location', 'Food and drink', 'Online boarding', 'Seat comfort',
       'Inflight entertainment', 'On-board service', 'Leg room service',
       'Baggage handling', 'Checkin service', 'Inflight service',
       'Cleanliness', 'Departure Delay in Minutes', 'Arrival Delay in Minutes',
       'satisfaction'],
      dtype='object')

In [128]:
ct = ColumnTransformer(transformers=[
    ("gender", OneHotEncoder(drop='first'), ["Gender", "Customer Type", "Type of Travel", "Class"]),
    ("scales", MinMaxScaler(), ["Age", "Flight Distance", "Inflight wifi service",
                                  "Departure/Arrival time convenient", "Ease of Online booking", "Gate location", "Food and drink", 'Online boarding', 'Seat comfort',
                                  'Inflight entertainment', 'On-board service', 'Leg room service',
                                  'Baggage handling', 'Checkin service', 'Inflight service',
                                  'Cleanliness', 'Departure Delay in Minutes', 'Arrival Delay in Minutes'])
], remainder="passthrough")

In [146]:
pipeline = Pipeline(steps=[
    ("convert columns", ct),
    ("imputer", SimpleImputer(strategy="mean"))
  ])

In [147]:
dfc = df.drop(columns=["id", "satisfaction"])
dfc = pipeline.fit_transform(dfc)

In [148]:
dfc = pd.DataFrame(dfc, columns=list(pipeline.named_steps["convert columns"].named_transformers_["gender"].get_feature_names_out())+["Age", "Flight Distance", "Inflight wifi service",
                                  "Departure/Arrival time convenient", "Ease of Online booking", "Gate location", "Food and drink", 'Online boarding', 'Seat comfort',
                                  'Inflight entertainment', 'On-board service', 'Leg room service',
                                  'Baggage handling', 'Checkin service', 'Inflight service',
                                  'Cleanliness', 'Departure Delay in Minutes', 'Arrival Delay in Minutes'])

In [149]:
dfc.head()

,Gender_Male,Customer Type_disloyal Customer,Type of Travel_Personal Travel,Class_Eco,Class_Eco Plus,Age,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,...,Seat comfort,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes
0,1.0,1.0,0.0,1.0,0.0,0.356164,0.109047,0.4,0.6,0.4,...,0.8,0.8,0.50,0.2,0.5,0.8,0.50,0.8,0.010858,0.003247
1,0.0,0.0,0.0,0.0,0.0,0.575342,0.282714,0.8,0.2,0.8,...,0.8,1.0,1.00,1.0,1.0,0.6,1.00,0.6,0.000000,0.000000
2,0.0,0.0,0.0,1.0,0.0,0.493151,0.168821,0.2,0.8,0.8,...,0.6,0.2,0.00,0.2,0.0,0.2,0.00,0.4,0.000000,0.019481
3,0.0,0.0,0.0,0.0,0.0,0.273973,0.306745,0.6,0.6,0.6,...,0.4,0.4,0.75,0.8,1.0,1.0,0.75,0.4,0.000000,0.000000
4,1.0,0.0,1.0,1.0,0.0,0.054795,0.099960,0.6,0.8,0.6,...,0.8,0.8,1.00,0.4,1.0,0.6,1.00,0.8,0.000000,0.010823


In [150]:
df.head()

,id,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,...,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,1,Male,disloyal Customer,33,Business travel,Eco,571,2,3,2,...,4,3,1,3,4,3,4,10,3.0,0
1,2,Female,Loyal Customer,49,Business travel,Business,1431,4,1,4,...,5,5,5,5,3,5,3,0,0.0,1
2,3,Female,Loyal Customer,43,Business travel,Eco,867,1,4,4,...,1,1,1,1,1,1,2,0,18.0,0
3,4,Female,Loyal Customer,27,Business travel,Business,1550,3,3,3,...,2,4,4,5,5,4,2,0,0.0,1
4,5,Male,Loyal Customer,11,Personal Travel,Eco,526,3,4,3,...,4,5,2,5,3,5,4,0,10.0,0


**modeling**

In [151]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_absolute_error, mean_squared_error

In [152]:
x_train, x_test, y_train, y_test = train_test_split(dfc, df["satisfaction"], test_size=0.2, random_state=42)

In [157]:
def loss(model_name, model):
    pred = model.predict(x_test)
    acr = accuracy_score(y_test, pred)
    mae = mean_absolute_error(y_test, pred)
    mse = mean_squared_error(y_test, pred)
    print('--', model_name, '--')
    print("ACR: ", acr*100, '%', sep='')
    print("MAE:", mae)
    print("MSE:", mse)
    print()

In [160]:
rf_model = RandomForestClassifier()
rf_model.fit(x_train, y_train)
loss('RandomForestClassifier', rf_model)


gb_model = GradientBoostingClassifier()
gb_model.fit(x_train, y_train)
loss('GradientBoostingClassifier', gb_model)


ab_model = AdaBoostClassifier()
ab_model.fit(x_train, y_train)
loss('AdaBoostClassifier', ab_model)


nb_model = BernoulliNB()
nb_model.fit(x_train, y_train)
loss('BernoulliNB', nb_model)


svc_model = SVC()
svc_model.fit(x_train, y_train)
loss('SVC', svc_model)


lr_model = LogisticRegression()
lr_model.fit(x_train, y_train)
loss('LogisticRegression', lr_model)


dt_model = DecisionTreeClassifier()
dt_model.fit(x_train, y_train)
loss('DecisionTreeClassifier', dt_model)


knn_model = KNeighborsClassifier()
knn_model.fit(x_train, y_train)
loss('KNeighborsClassifier', knn_model)


-- RandomForestClassifier --
ACR: 94.55%
MAE: 0.0545
MSE: 0.0545

-- GradientBoostingClassifier --
ACR: 93.8%
MAE: 0.062
MSE: 0.062



/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


-- AdaBoostClassifier --
ACR: 92.35%
MAE: 0.0765
MSE: 0.0765

-- BernoulliNB --
ACR: 79.95%
MAE: 0.2005
MSE: 0.2005

-- SVC --
ACR: 93.2%
MAE: 0.068
MSE: 0.068

-- LogisticRegression --
ACR: 87.6%
MAE: 0.124
MSE: 0.124

-- DecisionTreeClassifier --
ACR: 92.85%
MAE: 0.0715
MSE: 0.0715

-- KNeighborsClassifier --
ACR: 90.10000000000001%
MAE: 0.099
MSE: 0.099

